In [1]:
# 3rd Party Imports
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt 
import numpy as np
import matplotlib as plt
import geopandas as gpd
from fuzzywuzzy import fuzz, process

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

c:\Users\Babu\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Import eqao, osslt, and highschool data
current_dir = Path.cwd()
parent_dir = current_dir.parent

eqao = pd.read_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "EQAO_final.csv")
osslt = pd.read_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "OSSLT_final.csv")
high = pd.read_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "Highschool_final.csv")

In [3]:
merged_df = pd.merge(eqao, osslt, on='School Name')

merged_df

,School Name,EQAO % at Levels 3/4,OSSLT % Successful
0,A Y Jackson Secondary School,93.0,88.0
1,Agincourt Collegiate Institute,73.0,94.0
2,Albert Campbell Collegiate Institute,63.0,88.0
3,Birchmount Park Collegiate Institute,32.0,79.0
4,Bloor Collegiate Institute,79.0,94.0
...,...,...,...
59,Wexford Collegiate School for the Arts,42.0,86.0
60,Winston Churchill Collegiate Institute,20.0,70.0
61,Woburn Collegiate Institute,59.0,82.0
62,York Memorial Collegiate Institute,33.0,83.0


In [4]:
# investigate issues

merged_schools = merged_df['School Name'].str.lower()
high_schools = high['School Name'].str.lower()

# Find common schools (partial match in high within full names in merged_df)
common_schools = []
for partial_name in high_schools:
    matches = merged_schools[merged_schools.str.contains(partial_name, case=False, na=False)]
    common_schools.extend(matches.tolist())

# Find schools in merged_df but not in high
merged_only_schools = merged_schools[~merged_schools.isin(common_schools)].tolist()

# Find schools in high but not in merged_df (these are the partial names with no full match)
high_only_schools = high_schools[~high_schools.isin([name.lower() for name in common_schools])].tolist()

# Print the results
print("Common Schools (partial match in high within full names in merged_df):", common_schools)
print("Schools only in merged_df:", merged_only_schools)
print("Schools only in high:", high_only_schools)

Common Schools (partial match in high within full names in merged_df): ['a y jackson secondary school', 'agincourt collegiate institute', 'albert campbell collegiate institute', 'birchmount park collegiate institute', 'bloor collegiate institute', 'c w jefferys collegiate institute', 'cedarbrae collegiate institute', 'central technical school', 'central toronto academy', 'don mills collegiate institute', 'downsview secondary school', 'dr norman bethune collegiate institute', 'earl haig secondary school', 'east york collegiate institute', 'etobicoke collegiate institute', 'etobicoke school of the arts', 'forest hill collegiate institute', 'george s henry academy', 'georges vanier secondary school', 'harbord collegiate institute', 'humberside collegiate institute', 'jarvis collegiate institute', 'john polanyi collegiate institute', 'kipling collegiate institute', "l'amoreaux collegiate institute", 'lakeshore collegiate institute', 'lawrence park collegiate institute', 'leaside high schoo

In [5]:
# Problem Schools
# Greenwood Secondary School: Not included as only for Immigrant Students
# Heydon Park Secondary School: Not included as only for Young Women, Transgender and Non-Binary Students
# Native Learning Centre East: Not included as only for Indigeneous Students
# Rosedale Heights School of the Arts:
# SATEC @ WA Porter Collegiate Institute:
# Wexford Collegiate School for the Arts:

In [6]:
# merge osslt and eqao with fraiser ranking csv

# Extract the 'School Name' columns
merged_schools = merged_df['School Name'].str.lower()  # Convert to lowercase for case-insensitive comparison
high_schools = high['School Name'].str.lower()  # Convert to lowercase for case-insensitive comparison

high_schools = high_schools.tolist() 

# Find common schools (partial match in high within full names in merged_df)
common_schools = []
for partial_name in high_schools:
    matches = merged_schools[merged_schools.str.contains(partial_name, case=False, na=False)]
    common_schools.extend(matches.tolist())

# Find schools in merged_df but not in high
merged_only_schools = merged_schools[~merged_schools.isin(common_schools)].tolist()

# Find schools in high but not in merged_df (these are the partial names with no full match)
# We need to check that each name in high_schools is not in the common_schools
high_only_schools = [name for name in high_schools if name.lower() not in [s.lower() for s in common_schools]]

# Add a new "Partial Name" column in merged_df
merged_df['Partial Name'] = merged_df['School Name'].apply(lambda x: next((name for name in high_schools if name.lower() in x.lower()), None))

# Add a new "Partial Name" column in high
high['Partial Name'] = high['School Name'].apply(lambda x: next((name for name in merged_df['School Name'] if name.lower() in x.lower()), None))


In [7]:
# fix problem schools

special_schools = ['rosedale heights-arts', 'w a porter', 'wexford collegiate-arts']
merged_df.loc[merged_df['School Name'] == 'Rosedale Heights School of the Arts', 'Partial Name'] = 'rosedale heights-arts'
merged_df.loc[merged_df['School Name'] == 'SATEC @ WA Porter Collegiate Institute', 'Partial Name'] = 'w a porter'
merged_df.loc[merged_df['School Name'] == 'Wexford Collegiate School for the Arts', 'Partial Name'] = 'wexford collegiate-arts'

In [8]:
# Ensure both columns are strings and convert to lowercase, handling NaNs
merged_df["Partial Name"] = merged_df["Partial Name"].fillna("").astype(str).str.lower()
high["School Name"] = high["School Name"].fillna("").astype(str).str.lower()

# Perform a left merge, keeping all rows from merged_df and adding matching rows from high
merged_result = pd.merge(merged_df, high, how='left', left_on='Partial Name', right_on='School Name')

# Optional: Drop the 'School Name' column from the merged result if you don't need it
# merged_result = merged_result.drop(columns=['School Name'])

In [9]:
# clean columns 
merged_result = merged_result.drop(columns=['Partial Name_y', 'Partial Name_x', 'School Name_y'])
merged_result = merged_result.dropna()

merged_result = merged_result.rename(columns={'School Name_x': 'School Name'})
merged_result


,School Name,EQAO % at Levels 3/4,OSSLT % Successful,Fraiser Score
0,A Y Jackson Secondary School,93.0,88.0,7.9
1,Agincourt Collegiate Institute,73.0,94.0,8.3
2,Albert Campbell Collegiate Institute,63.0,88.0,7.0
3,Birchmount Park Collegiate Institute,32.0,79.0,5.2
4,Bloor Collegiate Institute,79.0,94.0,8.5
...,...,...,...,...
59,Wexford Collegiate School for the Arts,42.0,86.0,6.6
60,Winston Churchill Collegiate Institute,20.0,70.0,3.4
61,Woburn Collegiate Institute,59.0,82.0,6.9
62,York Memorial Collegiate Institute,33.0,83.0,6.7


In [10]:
# Export to CSV
over_ride = False
if over_ride:
    merged_result.to_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "Combined_schools_final.csv", index = False)

In [ ]:
# import schools with coords
gps_school = pd.read_csv(parent_dir / "2_data_cleaning" / "cleaned_csv" / "Combined_schools_with_coords.csv")
gps_school = gps_school.dropna(subset=["lat", "long"])

In [48]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gps_school['long'], gps_school['lat'])]
gdf = gpd.GeoDataFrame(gps_school, geometry=geometry)

gdf.set_crs("EPSG:4326", allow_override=True, inplace=True)

m = folium.Map(location=[43.7, -79.4], zoom_start=11)

def get_color(value, column):
    if column == "EQAO % at Levels 3/4":
        if value <= 20:
            return 'red'
        elif value <= 40:
            return 'orange'
        elif value <= 60:
            return 'yellow'
        elif value <= 80:
            return 'green'
        else:
            return 'blue'
    elif column == "OSSLT % Successful":
        if value <= 50:
            return 'red'
        elif value <= 70:
            return 'orange'
        elif value <= 85:
            return 'yellow'
        elif value <= 95:
            return 'green'
        else:
            return 'blue'
    elif column == "Fraiser Score":
        if value <= 3:
            return 'red'
        elif value <= 5:
            return 'orange'
        elif value <= 7:
            return 'yellow'
        elif value <= 8:
            return 'green'
        else:
            return 'blue'
    return 'gray'

def add_circle_markers(map_obj, gdf, column, layer_name):
    layer = folium.FeatureGroup(name=layer_name)
    for idx, row in gdf.iterrows():
        value = row[column]
        if pd.notna(value):
            color = get_color(value, column)
            folium.CircleMarker(
                location=[row['lat'], row['long']],
                radius=8,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.6,
                popup=f"{row['School Name']}<br>{column}: {value}",
                tooltip=f"{row['School Name']}<br>{column}: {value}"
            ).add_to(layer)
    layer.add_to(map_obj)

add_circle_markers(m, gdf, "EQAO % at Levels 3/4", "EQAO % at Levels 3/4")
add_circle_markers(m, gdf, "OSSLT % Successful", "OSSLT % Successful")
add_circle_markers(m, gdf, "Fraiser Score", "Fraiser Score")

folium.LayerControl().add_to(m)

def add_legend(map_obj, column):
    if column == "EQAO % at Levels 3/4":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 50px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>EQAO % at Levels 3/4 Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-20<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 21-40<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 41-60<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 61-80<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 81-100<br>
            </div>
        """
    elif column == "OSSLT % Successful":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 220px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>OSSLT % Successful Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-50<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 51-70<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 71-85<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 86-95<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 96-100<br>
            </div>
        """
    elif column == "Fraiser Score":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 390px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>Fraiser Score Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-3<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 4-5<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 6-7<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 8<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 9-10<br>
            </div>
        """
    
    map_obj.get_root().html.add_child(folium.Element(legend))

add_legend(m, "EQAO % at Levels 3/4")
add_legend(m, "OSSLT % Successful")
add_legend(m, "Fraiser Score")

m


In [47]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from folium.plugins import HeatMap

geometry = [Point(xy) for xy in zip(gps_school['long'], gps_school['lat'])]
gdf = gpd.GeoDataFrame(gps_school, geometry=geometry)

gdf.set_crs("EPSG:4326", allow_override=True, inplace=True)

m = folium.Map(location=[43.7, -79.4], zoom_start=11)

def get_color(value, column):
    if column == "EQAO % at Levels 3/4":
        if value <= 20:
            return 'red'
        elif value <= 40:
            return 'orange'
        elif value <= 60:
            return 'yellow'
        elif value <= 80:
            return 'green'
        else:
            return 'blue'
    elif column == "OSSLT % Successful":
        if value <= 50:
            return 'red'
        elif value <= 70:
            return 'orange'
        elif value <= 85:
            return 'yellow'
        elif value <= 95:
            return 'green'
        else:
            return 'blue'
    elif column == "Fraiser Score":
        if value <= 3:
            return 'red'
        elif value <= 5:
            return 'orange'
        elif value <= 7:
            return 'yellow'
        elif value <= 8:
            return 'green'
        else:
            return 'blue'
    return 'gray'

def add_heatmap(map_obj, gdf, column, layer_name):
    heat_data = []
    for idx, row in gdf.iterrows():
        value = row[column]
        if pd.notna(value):
            lat, long = row['lat'], row['long']
            color = get_color(value, column)  # Get color for intensity
            # Here we use the value as the intensity for the heatmap
            heat_data.append([lat, long, value])
    
    HeatMap(heat_data, name=layer_name, radius=15, max_zoom=13).add_to(map_obj)

add_heatmap(m, gdf, "EQAO % at Levels 3/4", "EQAO % at Levels 3/4")
add_heatmap(m, gdf, "OSSLT % Successful", "OSSLT % Successful")
add_heatmap(m, gdf, "Fraiser Score", "Fraiser Score")

folium.LayerControl().add_to(m)

def add_legend(map_obj, column):
    if column == "EQAO % at Levels 3/4":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 50px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>EQAO % at Levels 3/4 Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-20<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 21-40<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 41-60<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 61-80<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 81-100<br>
            </div>
        """
    elif column == "OSSLT % Successful":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 220px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>OSSLT % Successful Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-50<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 51-70<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 71-85<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 86-95<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 96-100<br>
            </div>
        """
    elif column == "Fraiser Score":
        legend = """
            <div style="position: fixed; 
                        bottom: 50px; left: 390px; width: 160px; height: 140px; 
                        background-color: white; z-index: 9999; font-size: 12px;
                        border-radius: 5px; padding: 10px; box-shadow: 0px 0px 15px rgba(0,0,0,0.2);">
                <b>Fraiser Score Legend</b><br>
                <i style="background: red; width: 12px; height: 12px; display: inline-block;"></i> 0-3<br>
                <i style="background: orange; width: 12px; height: 12px; display: inline-block;"></i> 4-5<br>
                <i style="background: yellow; width: 12px; height: 12px; display: inline-block;"></i> 6-7<br>
                <i style="background: green; width: 12px; height: 12px; display: inline-block;"></i> 8<br>
                <i style="background: blue; width: 12px; height: 12px; display: inline-block;"></i> 9-10<br>
            </div>
        """
    
    map_obj.get_root().html.add_child(folium.Element(legend))

add_legend(m, "EQAO % at Levels 3/4")
add_legend(m, "OSSLT % Successful")
add_legend(m, "Fraiser Score")

m
